<a href="https://colab.research.google.com/github/CasCard/Machine-Learning-Project/blob/master/MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
from six.moves import cPickle as pickle
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import tensorflow as tf
from tensorflow import keras

In [0]:
mnist=tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()


In [0]:
train_dataset=x_train.reshape([x_train.shape[0],x_train.shape[1],x_train.shape[2],1])
test_dataset=x_test.reshape([x_test.shape[0],x_test.shape[1],x_test.shape[2],1])

In [4]:
print(y_train.shape)
print(x_train.shape)
print(x_test.shape)
print(y_test.shape)
print(train_dataset.shape)
print(test_dataset.shape)


(60000,)
(60000, 28, 28)
(10000, 28, 28)
(10000,)
(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [0]:
def One_hot_encoder(labels,C):
    C=tf.constant(C,dtype=tf.int32,name='C')
    encoded=tf.one_hot(labels,C)
    with tf.Session() as sess:
        return sess.run(encoded)

In [6]:
Train_labels=One_hot_encoder(y_train,10)
Test_labels=One_hot_encoder(y_test,10)
print(Train_labels.shape)

(60000, 10)


In [0]:
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    X = tf.placeholder(shape=(None, n_H0, n_W0, n_C0),dtype=tf.float32)
    Y = tf.placeholder(shape=(None, n_y),dtype=tf.float32)
    return X, Y

In [0]:
def initialize_parameters():
    W1 = tf.get_variable('W1',[4,4,1,8],initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W2 = tf.get_variable('W2',[2,2,8,16],initializer=tf.contrib.layers.xavier_initializer(seed=0))
    parameters = {"W1": W1,
                  "W2": W2}
    return parameters

In [0]:
def forward_propagation(X, parameters):
    W1 = parameters['W1']
    W2 = parameters['W2']
    Z1 = tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')
    A1 = tf.nn.relu(Z1)
    P1 = tf.nn.max_pool(A1,padding='SAME',ksize=[1,8,8,1],strides=[1,8,8,1])
    Z2 = tf.nn.conv2d(P1,W2,strides=[1,1,1,1],padding='SAME')
    A2 = tf.nn.relu(Z2)
    P2 = tf.nn.max_pool(A2,padding='SAME',ksize=[1,4,4,1],strides=[1,4,4,1])
    P2 = tf.contrib.layers.flatten(P2)
    Z3 = tf.contrib.layers.fully_connected(P2,10,activation_fn=None)
    return Z3

In [0]:
def compute_cost(Z3, Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y))
    return cost

In [11]:
import time as t
X, Y = create_placeholders(28,28,1,10)
parameters = initialize_parameters()
Z3 = forward_propagation(X, parameters)
cost = compute_cost(Z3, Y)
optimizer = tf.train.AdamOptimizer().minimize(cost)
init = tf.global_variables_initializer()
predict_op = tf.argmax(Z3, 1)
correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))        
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
with tf.Session() as sess:
    sess.run(init)
    print("Training Started")
    A=t.time()
    for i in range(500):
        _ , temp_cost,acc = sess.run([optimizer,cost,accuracy],{X:train_dataset,Y:Train_labels})
        print(i," : ",temp_cost," : ",acc)
    print('Training Over time taken ',t.time()-A,'sec')
    A=t.time()
    print('Testing Accuracy',sess.run(accuracy,{X:test_dataset,Y:Test_labels}))
    print('Testing Time',t.time()-A)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Training Started
0  :  141.3339  :  0.11906666
1  :  129.22856  :  0.13741666
2  :  119.209366  :  0.15423334
3  :  111.204216  :  0.16031666
4  :  104.57083  :  0.1599
5  :  98.6723  :  0.15908334
6  :  92.95895  :  0.16078334
7  :  87.167725  :  0.16411667
8  :  81.33274  :  0.16726667
9  :  75.65827  :  0.168